In [1]:
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, plot_roc_curve, make_scorer, f1_score, roc_auc_score
from sklearn import preprocessing
from scipy import stats
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate, LeaveOneGroupOut, PredefinedSplit, GridSearchCV
import matplotlib.pyplot as plt
import os
import json

%matplotlib inline

In [2]:
def concat_dataframes(path, df_type):
    dfs_list = []
    dfs_rows_len_list = []
    
    for user in os.listdir(path):
        for file in os.listdir(os.path.join(path, user)):
            if file.find(df_type) != -1:
                df = pd.read_csv(os.path.join(path, user, file))
                
                if df_type != 'broadcasts':
                    df = df.drop(["timestamp"], axis=1)
#                 df = (df - df.min()) / (df.max() - df.min())
                
                df["user"] = int(user.split('_')[1])
                
                dfs_list.append(df)
    
    return pd.concat(dfs_list, ignore_index=True)


def drop_bad_rows(df, z = 3):
    bad_rows = set()
    for col in df.columns:
        if col != "user":
            for user in df.user.unique():
                for x in list(df.loc[df.user == user, :][np.abs(stats.zscore(df.loc[df.user == user, col])) > z].index):
                    bad_rows.add(x)

            for x in list(df[col][np.abs(stats.zscore(df[col])) > z].index):
                bad_rows.add(x)

    df = df.drop(list(bad_rows), axis=0)

    return df


def drop_bad_cols(df, z = 3, allowed_proportion = 0.1):
    bad_cols = set()
    for col in df.columns:
        if col != "user":
            if df[df[col] != df[col].mean()].shape[0] < allowed_proportion * df.shape[0]:
                bad_cols.add(col)

            for user in df.user.unique():
                if df.loc[df.user == user, :][df.loc[df.user == user, col] != df.loc[df.user == user, col].mean()].shape[0] < allowed_proportion * df.loc[df.user == user, :].shape[0]:
                    bad_cols.add(col)

                elif np.sum(np.abs(stats.zscore(df.loc[df.user == user, col])) < z) < (1 - allowed_proportion) * df.loc[df.user == user, col].shape[0]:
                    bad_cols.add(col)

    df = df.drop(bad_cols, axis=1)
    return df, list(bad_cols)
    
    
def extract_delayed_user(df, user_label):
    df_user = df[df["user"] == user_label]
    df = df[df["user"] != user_label]
    return df_user, df


def split_users_into_two_classes(df, valid_user_label):
    df.loc[df["user"] != valid_user_label, "user"] = 0
    df.loc[df["user"] == valid_user_label, "user"] = 1
    return df  


def get_cv_split(X, y, group_labels, valid_user_label):
    predefined_split_array = np.zeros(group_labels.shape[0])
    i = 0
    test_array = [x for x in range(group_labels.shape[0])]
    for test, _ in LeaveOneGroupOut().split(X, y, group_labels):
        diff = np.setdiff1d(test_array, test)
        if np.all(group_labels[diff[0] : diff[-1]] == valid_user_label) is np.bool_(True):
            for sample in diff:
                predefined_split_array[sample] = -1
        else:
            for sample in diff:
                predefined_split_array[sample] = i
            i += 1
    return predefined_split_array


def generate_train_dataset(df, user, ex_user, is_SVM = False):
    df_ = df.copy()

    df_for_test = []

    df__ = df_[df_.labels == ex_user].copy()
    df_for_test.append(df__)
    df_ = df_.drop(df__.index, axis=0)

    for user_ in df_.labels.unique():
        if user_ != ex_user:
            test_size = int((0.25 * df_[df_.labels == user_].shape[0]) - 1)
            df__ = df_[df_.labels == user_].sample(test_size).copy()
            df_for_test.append(df__)
            df_ = df_.drop(df__.index, axis=0)

    df_ = split_users_into_two_classes(df_.copy(), user)
          
    if is_SVM:    
        df_.loc[df_.user == 0, 'user'] = -1

    df_ = df_.drop("labels", axis=1)

    dataset = df_.to_numpy().copy()
    np.random.shuffle(dataset)

    X = dataset[:, :-1]
    y = dataset[:, -1]
    
    return X, y, df_for_test


def generate_test_dataset(df_list, user, ex_user, is_SVM = False):
    test_df = pd.concat(df_list)

    valid_user_in_test_count = test_df[test_df.labels == user].shape[0]
    ex_user_in_test_count = test_df[test_df.labels == ex_user].shape[0]
    others_in_test_count = [test_df[test_df.labels == x].shape[0]
                            for x in test_df.labels.unique() if x != user and x != ex_user]

    others_test_count = sum(others_in_test_count)
    part_size = min(valid_user_in_test_count, ex_user_in_test_count)
    if others_test_count <= min(valid_user_in_test_count, ex_user_in_test_count):
        part_size = others_test_count    
        
    new_df_parts = []    

    new_df_parts.append(test_df[test_df.labels == user].sample(part_size).copy())
    new_df_parts.append(test_df[test_df.labels == ex_user].sample(part_size).copy())
    new_df_parts.append(test_df[~test_df.labels.isin([user, ex_user])].sample(part_size).copy())
    
    test_df = pd.concat(new_df_parts)
    
    test_df.loc[test_df.labels == user, "user"] = 1
    if is_SVM:
        test_df.loc[test_df.labels != user, "user"] = -1
    else:
        test_df.loc[test_df.labels != user, "user"] = 0

    print("True: ", test_df[test_df.user == 1].shape)
    print("Shape: ", test_df.shape)
    for x in test_df.labels.unique():
        print("Count ", x, ": ", test_df[test_df.labels == x].shape)

    test_df = test_df.drop("labels", axis=1)

    test_dataset = test_df.to_numpy().copy()
    X_test = test_dataset[:, :-1].copy()
    y_test = test_dataset[:, -1].copy()

    return X_test, y_test


def prepare_dataset(df, user, is_SVM=False):
    df_ = split_users_into_two_classes(df.copy(), user)
    
    group_labels = df_.labels.to_numpy().copy()
    df_ = df_.drop('labels', axis=1)
    
    if is_SVM:
        df_.loc[df_.user == 0, 'user'] = -1
    
    dataset = df_.to_numpy().copy()
    X = dataset[:, :-1]
    y = dataset[:, -1]
    
    return X, y, group_labels


def create_file_for_results(data_type):
    res_folder = '.\\_results'
    if os.path.exists(res_folder) is False:
        os.makedirs(res_folder)
    
    file = os.path.join(res_folder, data_type + '_results.json')
    if os.path.exists(file) is False:
        with open(file, 'w') as f:
            json.dump({'stub': None}, f)
        
    return file    


def update_file_with_results(file_path, results_dict):
    import collections.abc

    def update(d, u):
        for k, v in u.items():
            if isinstance(v, collections.abc.Mapping):
                d[k] = update(d.get(k, {}), v)
            else:
                d[k] = v
        return d
    
    with open(file_path, 'r') as f:
        res = json.load(f)
    
    res = update(res, results_dict)
    
    with open(file_path, 'w') as f:
        json.dump(res, f, sort_keys=True, indent=2)
        
        
def get_dict_with_results(json_path):
    with open(json_path, 'r') as f:
        res = json.load(f)
    return res    


def get_dataframe(path, data_type, window_type, window_size):
    return concat_dataframes(os.path.join(path, window_type, window_size), data_type), create_file_for_results(data_type)


def drop_corr_columns(df, corr_coef):
    corr_matrix = df.corr().abs()
    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    corr_cols = [column for column in upper_tri.columns if any(abs(upper_tri[column]) > corr_coef) and column != "user"]
    return df.drop(corr_cols, axis=1), corr_cols


def process_train_df(df, features, corr = 0.7, z = 3, prop = 0.1): 
    df = df.drop(df.columns.difference(features), axis=1)
    df = df.dropna(how='all')
    df = df.fillna(0)
    
    if 'count_mean' in df.columns:
        df = df[df.count_mean != 0]
    
    df = drop_bad_rows(df, z)
    df, dropped_cols_1 = drop_bad_cols(df, z, prop)
    df, dropped_cols_2 = drop_corr_columns(df, corr)
    
    return df, dropped_cols_1 + dropped_cols_2

In [3]:
def model_cross_validation(results_file, model, df, model_tag, df_type, window_type, window_size, is_SVM = False):
    for user in df.labels.unique():
        print("Valid User: ", user)
        print("--------------------------------------------------------------------------------")

        X, y, group_labels = prepare_dataset(df, user, is_SVM)

        cv_split = PredefinedSplit(test_fold=get_cv_split(X, y, group_labels, user))
        scoring = ('accuracy')

        cv_results = cross_validate(model, X, y, scoring=scoring, cv=cv_split, n_jobs=-1)
        accuracy = cv_results['test_score']
        
        results = {
            df_type: {
                window_type: {
                    window_size: {
                        model_tag: {
                            "cross_validation": {
                                "valid_user": {
                                    str(user): {
                                        "accuracy": accuracy.tolist()
                                    }
                                }
                            }
                        }
                    }
                }
            }
        }
        
        update_file_with_results(results_file, results)

        print("CV accuracy list: ", accuracy)
        print("CV mean accuracy: ", np.mean(accuracy))
        print("CV min accuracy: ", np.min(accuracy))
        print("CV max accuracy: ", np.max(accuracy))

        print("--------------------------------------------------------------------------------")
        
        
def model_final_validation(results_file, model, df, model_tag, df_type, window_type, window_size, is_SVM = False):
    for user in df.labels.unique():
        print("Valid User: ", user)
        print("--------------------------------------------------------------------------------")
        for ex_user in df.labels.unique():
            if ex_user != user:
                X, y, df_for_test = generate_train_dataset(df, user, ex_user, is_SVM)

                model.fit(X, y)

                X_test, y_test = generate_test_dataset(df_for_test, user, ex_user, is_SVM)

                predict = model.predict(X_test)
#                 if is_SVM:
#                     proba = model.decision_function(X_test)
#                 else:
                proba = model.predict_proba(X_test)

                results = {
                    df_type: {
                        window_type: {
                            window_size: {
                                model_tag: {
                                    "final_validation": {
                                        "valid_user": {
                                            str(user): {
                                                "extracted_user": {
                                                    str(ex_user): {
                                                        "test": y_test.tolist(),
                                                        "predict": predict.tolist(),
                                                        "proba": proba.tolist()
                                                    }
                                                }
                                            }
                                        }
                                    }
                                }
                            }
                        }
                    }
                }

                update_file_with_results(results_file, results)

                print("Valid user = ", user, ", Extracted user = ", ex_user, "accuracy = ", accuracy_score(y_test, predict))
                print("--------------------------------------------------------------------------------")     

### Learning settings
### ***

In [4]:
DATA_PATH = '..\\..\\scripts\\_features_all'

DATA_TYPE = "bt"

WINDOW_TYPE = "rolling"
WINDOW_SIZE = "60s"

In [5]:
DATA_TYPES = ['wifi', 'bt', 'location']
WINDOW_TYPES = ['rolling', 'sampling']
# 60s, 
# WINDOWS = ['5s', '10s', '30s', '60s', '90s', '120s', '240s', '600s']
WINDOWS = ['90s', '120s']

In [6]:
catboost_params = {
    'iterations': 100,
    'depth': 6,
    'loss_function': 'Logloss',
    'l2_leaf_reg': 1,
    'leaf_estimation_iterations': 5,
    'logging_level': 'Silent'
}

randomforest_params = {
    'n_estimators': 100,
    'criterion': 'gini',
    'max_depth': None,
    'min_samples_split': 2,
    'min_samples_leaf': 1,
    'max_features': 'auto',
    'n_jobs': -1,
    'class_weight': 'balanced',
}

svc_params = {
    'C': 1,
    'kernel': 'rbf',
    'degree': 1,
    'gamma': 5,
    'probability': True
}

logreg_params = {
    'penalty': 'l2',
    'C': 0.01,
    'solver': 'newton-cg',
    'max_iter': 1000,
    'n_jobs': -1
}

MODELS = [
    (CatBoostClassifier(**catboost_params), "CatBoost"),
    (RandomForestClassifier(**randomforest_params), "RandomForest"),
#     (SVC(**svc_params), "SVC"),
    (LogisticRegression(**logreg_params), "LogReg")
]

### ***

## Cross-validation

In [7]:
for data_type in DATA_TYPES:
    for wnd_type in WINDOW_TYPES:
        for wnd in WINDOWS:
            df, RESULTS_FILE = get_dataframe(DATA_PATH, data_type, wnd_type, wnd)
            features = df.columns.to_list()
            df, _ = process_train_df(df, features)
            df['labels'] = df['user']
            
            for model, tag in MODELS:
                print(data_type, wnd_type, wnd, tag)
                model_cross_validation(RESULTS_FILE, model, df, tag, data_type, wnd_type, wnd, is_SVM=tag=='SVC')

D:\Program Files\Anaconda3\lib\site-packages\scipy\stats\stats.py:2500: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in greater


wifi rolling 90s CatBoost
Valid User:  1
--------------------------------------------------------------------------------
CV accuracy list:  [0.52570093 0.99039781 0.93465279 0.83942344 0.90229885 0.99603175
 0.78241758]
CV mean accuracy:  0.8529890214415997
CV min accuracy:  0.5257009345794392
CV max accuracy:  0.996031746031746
--------------------------------------------------------------------------------
Valid User:  2
--------------------------------------------------------------------------------
CV accuracy list:  [0.98303778 0.96844993 0.99163965 0.98956885 1.         0.99206349
 1.        ]
CV mean accuracy:  0.9892513857889949
CV min accuracy:  0.9684499314128944
CV max accuracy:  1.0
--------------------------------------------------------------------------------
Valid User:  3
--------------------------------------------------------------------------------
CV accuracy list:  [0.99672321 0.98598131 0.99829381 0.99032748 0.8908046  0.84920635
 0.71208791]
CV mean accuracy:  

CV accuracy list:  [1.         0.80919003 0.99725652 0.95290906 0.99810343 1.
 0.98901099]
CV mean accuracy:  0.9637814317574614
CV min accuracy:  0.809190031152648
CV max accuracy:  1.0
--------------------------------------------------------------------------------
Valid User:  8
--------------------------------------------------------------------------------
CV accuracy list:  [0.99518119 0.99844237 0.95061728 1.         0.99917815 0.67241379
 0.73809524]
CV mean accuracy:  0.9077040030827129
CV min accuracy:  0.6724137931034483
CV max accuracy:  1.0
--------------------------------------------------------------------------------


D:\Program Files\Anaconda3\lib\site-packages\scipy\stats\stats.py:2500: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in greater


wifi rolling 120s CatBoost
Valid User:  1
--------------------------------------------------------------------------------
CV accuracy list:  [0.63671572 0.98026316 0.96371031 0.85366641 0.78698225 0.99423631
 0.67070707]
CV mean accuracy:  0.8408973184607011
CV min accuracy:  0.6367157242447715
CV max accuracy:  0.9942363112391931
--------------------------------------------------------------------------------
Valid User:  2
--------------------------------------------------------------------------------
CV accuracy list:  [0.98617512 0.92105263 0.994056   0.98657372 1.         0.98847262
 0.9979798 ]
CV mean accuracy:  0.9820442687798832
CV min accuracy:  0.9210526315789473
CV max accuracy:  1.0
--------------------------------------------------------------------------------
Valid User:  3
--------------------------------------------------------------------------------
CV accuracy list:  [0.99719495 0.91324555 0.99937432 0.99573974 0.92899408 0.86743516
 0.74747475]
CV mean accuracy:

...

CV accuracy list:  [1.         1.         1.         1.         1.         0.93929712
 1.        ]
CV mean accuracy:  0.9913281606572342
CV min accuracy:  0.939297124600639
CV max accuracy:  1.0
--------------------------------------------------------------------------------
Valid User:  8
--------------------------------------------------------------------------------
CV accuracy list:  [1. 1. 1. 1. 1. 1. 1.]
CV mean accuracy:  1.0
CV min accuracy:  1.0
CV max accuracy:  1.0
--------------------------------------------------------------------------------
location rolling 90s CatBoost
Valid User:  1
--------------------------------------------------------------------------------
CV accuracy list:  [0.99169475 0.70791434 0.98322742 0.875      0.98680312 0.98984865
 0.99368643]
CV mean accuracy:  0.9325963875860327
CV min accuracy:  0.7079143389199255
CV max accuracy:  0.9936864258155034
--------------------------------------------------------------------------------
Valid User:  2
-----

CV accuracy list:  [0.98262675 0.96665864 0.98035382 0.70554224 0.83746297 0.80490956
 0.99017888]
CV mean accuracy:  0.8953904093269305
CV min accuracy:  0.7055422439837483
CV max accuracy:  0.9901788846018941
--------------------------------------------------------------------------------
Valid User:  6
--------------------------------------------------------------------------------
CV accuracy list:  [0.9996722  0.95618681 0.99599628 0.96655902 0.90896092 0.97268365
 0.99964925]
CV mean accuracy:  0.9713868730001185
CV min accuracy:  0.9089609151572927
CV max accuracy:  0.9996722027972028
--------------------------------------------------------------------------------
Valid User:  7
--------------------------------------------------------------------------------
CV accuracy list:  [0.99825175 0.99987963 0.99757914 0.99229086 0.98522402 0.9989227
 1.        ]
CV mean accuracy:  0.9960211594645803
CV min accuracy:  0.9852240228789323
CV max accuracy:  1.0
-----------------------------

CV accuracy list:  [0.99084369 0.98523587 0.9995367  0.80737754 0.90123132 0.96343001
 1.        ]
CV mean accuracy:  0.9496650181434776
CV min accuracy:  0.8073775400215577
CV max accuracy:  1.0
--------------------------------------------------------------------------------
Valid User:  4
--------------------------------------------------------------------------------
CV accuracy list:  [0.98844561 0.99915976 1.         0.92626452 0.79851189 0.61844713
 0.96200767]
CV mean accuracy:  0.8989766533440589
CV min accuracy:  0.6184471266438479
CV max accuracy:  1.0
--------------------------------------------------------------------------------
Valid User:  5
--------------------------------------------------------------------------------
CV accuracy list:  [0.99258775 0.9549874  0.98768473 0.71327087 0.87620992 0.80742209
 0.99407459]
CV mean accuracy:  0.903748191505836
CV min accuracy:  0.7132708740862761
CV max accuracy:  0.994074590449634
---------------------------------------------

CV accuracy list:  [1.         1.         1.         0.98393267 0.99273608 1.
 1.        ]
CV mean accuracy:  0.9966669639598607
CV min accuracy:  0.9839326702371843
CV max accuracy:  1.0
--------------------------------------------------------------------------------
Valid User:  2
--------------------------------------------------------------------------------
CV accuracy list:  [1.         1.         1.         0.99693956 1.         1.
 1.        ]
CV mean accuracy:  0.9995627937479507
CV min accuracy:  0.9969395562356542
CV max accuracy:  1.0
--------------------------------------------------------------------------------
Valid User:  3
--------------------------------------------------------------------------------
CV accuracy list:  [1.         1.         1.         0.99540933 1.         1.
 1.        ]
CV mean accuracy:  0.9993441906219259
CV min accuracy:  0.9954093343534812
CV max accuracy:  1.0
--------------------------------------------------------------------------------
V

CV accuracy list:  [0.99433962 0.99596774 0.99621928 0.99891186 0.98377863 0.97746479
 0.99166667]
CV mean accuracy:  0.9911926554731344
CV min accuracy:  0.9774647887323944
CV max accuracy:  0.998911860718172
--------------------------------------------------------------------------------
Valid User:  8
--------------------------------------------------------------------------------
CV accuracy list:  [0.99056604 1.         0.97542533 0.82045702 0.99332061 1.
 0.99625468]
CV mean accuracy:  0.9680033827688621
CV min accuracy:  0.8204570184983678
CV max accuracy:  1.0
--------------------------------------------------------------------------------
location sampling 120s LogReg
Valid User:  1
--------------------------------------------------------------------------------
CV accuracy list:  [1.         1.         1.         0.97614504 0.9971831  1.
 1.        ]
CV mean accuracy:  0.996189733822784
CV min accuracy:  0.976145038167939
CV max accuracy:  1.0
--------------------------------

## Final Validation

In [ ]:
for data_type in DATA_TYPES:
    for wnd_type in WINDOW_TYPES:
        for wnd in WINDOWS:
            df, RESULTS_FILE = get_dataframe(DATA_PATH, data_type, wnd_type, wnd)
            features = df.columns.to_list()
            df, _ = process_train_df(df, features)
            df['labels'] = df['user']
                
            for model, tag in MODELS:
                print(data_type, wnd_type, wnd, tag)
                model_final_validation(RESULTS_FILE, model, df, tag, data_type, wnd_type, wnd, is_SVM=tag=='SVC')

D:\Program Files\Anaconda3\lib\site-packages\scipy\stats\stats.py:2500: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in greater


wifi rolling 90s CatBoost
Valid User:  1
--------------------------------------------------------------------------------
True:  (1284, 23)
Shape:  (3852, 23)
Count  1 :  (1284, 23)
Count  2 :  (1284, 23)
Count  5 :  (877, 23)
Count  3 :  (40, 23)
Count  4 :  (319, 23)
Count  6 :  (13, 23)
Count  8 :  (22, 23)
Count  7 :  (13, 23)
Valid user =  1 , Extracted user =  2 accuracy =  0.8011422637590861
--------------------------------------------------------------------------------
True:  (729, 23)
Shape:  (2187, 23)
Count  1 :  (729, 23)
Count  3 :  (729, 23)
Count  4 :  (182, 23)
Count  5 :  (489, 23)
Count  2 :  (37, 23)
Count  8 :  (13, 23)
Count  7 :  (5, 23)
Count  6 :  (3, 23)
Valid user =  1 , Extracted user =  3 accuracy =  0.9538180155464107
--------------------------------------------------------------------------------
True:  (1296, 23)
Shape:  (3888, 23)
Count  1 :  (1296, 23)
Count  4 :  (1296, 23)
Count  5 :  (1124, 23)
Count  8 :  (24, 23)
Count  3 :  (44, 23)
Count  2 :  (

Valid user =  4 , Extracted user =  2 accuracy =  0.952232606438214
--------------------------------------------------------------------------------
True:  (729, 23)
Shape:  (2187, 23)
Count  4 :  (729, 23)
Count  3 :  (729, 23)
Count  7 :  (8, 23)
Count  5 :  (479, 23)
Count  1 :  (168, 23)
Count  2 :  (46, 23)
Count  8 :  (18, 23)
Count  6 :  (10, 23)
Valid user =  4 , Extracted user =  3 accuracy =  0.9561042524005487
--------------------------------------------------------------------------------
True:  (1464, 23)
Shape:  (4392, 23)
Count  4 :  (1464, 23)
Count  5 :  (1464, 23)
Count  1 :  (939, 23)
Count  2 :  (232, 23)
Count  8 :  (83, 23)
Count  3 :  (134, 23)
Count  7 :  (45, 23)
Count  6 :  (31, 23)
Valid user =  4 , Extracted user =  5 accuracy =  0.7379326047358834
--------------------------------------------------------------------------------
True:  (174, 23)
Shape:  (522, 23)
Count  4 :  (174, 23)
Count  6 :  (174, 23)
Count  5 :  (122, 23)
Count  2 :  (11, 23)
Count  1 :

True:  (62, 23)
Shape:  (186, 23)
Count  7 :  (62, 23)
Count  5 :  (62, 23)
Count  8 :  (5, 23)
Count  1 :  (22, 23)
Count  4 :  (30, 23)
Count  3 :  (3, 23)
Count  2 :  (1, 23)
Count  6 :  (1, 23)
Valid user =  7 , Extracted user =  5 accuracy =  0.946236559139785
--------------------------------------------------------------------------------
True:  (62, 23)
Shape:  (186, 23)
Count  7 :  (62, 23)
Count  6 :  (62, 23)
Count  5 :  (33, 23)
Count  4 :  (16, 23)
Count  1 :  (9, 23)
Count  3 :  (3, 23)
Count  2 :  (1, 23)
Valid user =  7 , Extracted user =  6 accuracy =  0.9516129032258065
--------------------------------------------------------------------------------
True:  (62, 23)
Shape:  (186, 23)
Count  7 :  (62, 23)
Count  8 :  (62, 23)
Count  4 :  (18, 23)
Count  3 :  (4, 23)
Count  1 :  (6, 23)
Count  2 :  (4, 23)
Count  5 :  (30, 23)
Valid user =  7 , Extracted user =  8 accuracy =  0.9623655913978495
------------------------------------------------------------------------------

True:  (320, 23)
Shape:  (960, 23)
Count  2 :  (320, 23)
Count  8 :  (320, 23)
Count  5 :  (183, 23)
Count  1 :  (55, 23)
Count  4 :  (66, 23)
Count  3 :  (11, 23)
Count  6 :  (2, 23)
Count  7 :  (3, 23)
Valid user =  2 , Extracted user =  8 accuracy =  0.975
--------------------------------------------------------------------------------
Valid User:  3
--------------------------------------------------------------------------------
True:  (181, 23)
Shape:  (543, 23)
Count  3 :  (181, 23)
Count  1 :  (181, 23)
Count  2 :  (11, 23)
Count  5 :  (118, 23)
Count  4 :  (40, 23)
Count  7 :  (5, 23)
Count  8 :  (6, 23)
Count  6 :  (1, 23)
Valid user =  3 , Extracted user =  1 accuracy =  0.860036832412523
--------------------------------------------------------------------------------
True:  (181, 23)
Shape:  (543, 23)
Count  3 :  (181, 23)
Count  2 :  (181, 23)
Count  5 :  (95, 23)
Count  4 :  (47, 23)
Count  1 :  (32, 23)
Count  8 :  (6, 23)
Count  7 :  (1, 23)
Valid user =  3 , Extracted u

...

Valid user =  1 , Extracted user =  7 accuracy =  0.6455788248981966
--------------------------------------------------------------------------------
True:  (2292, 14)
Shape:  (6876, 14)
Count  1 :  (2292, 14)
Count  8 :  (2292, 14)
Count  5 :  (669, 14)
Count  6 :  (408, 14)
Count  2 :  (229, 14)
Count  3 :  (294, 14)
Count  4 :  (552, 14)
Count  7 :  (140, 14)
Valid user =  1 , Extracted user =  8 accuracy =  0.6671029668411868
--------------------------------------------------------------------------------
Valid User:  2
--------------------------------------------------------------------------------
True:  (2081, 14)
Shape:  (6243, 14)
Count  2 :  (2081, 14)
Count  1 :  (2081, 14)
Count  5 :  (661, 14)
Count  8 :  (80, 14)
Count  7 :  (153, 14)
Count  3 :  (291, 14)
Count  4 :  (488, 14)
Count  6 :  (408, 14)
Valid user =  2 , Extracted user =  1 accuracy =  0.6658657696620215
--------------------------------------------------------------------------------
True:  (2081, 14)
Shape: 

Valid user =  4 , Extracted user =  8 accuracy =  0.7312652492157546
--------------------------------------------------------------------------------
Valid User:  5
--------------------------------------------------------------------------------
True:  (6261, 14)
Shape:  (18783, 14)
Count  5 :  (6261, 14)
Count  1 :  (6261, 14)
Count  6 :  (1504, 14)
Count  3 :  (1108, 14)
Count  4 :  (1925, 14)
Count  7 :  (549, 14)
Count  2 :  (874, 14)
Count  8 :  (301, 14)
Valid user =  5 , Extracted user =  1 accuracy =  0.7100037267742108
--------------------------------------------------------------------------------
True:  (6261, 14)
Shape:  (18783, 14)
Count  5 :  (6261, 14)
Count  2 :  (6261, 14)
Count  4 :  (1933, 14)
Count  6 :  (1512, 14)
Count  3 :  (1092, 14)
Count  8 :  (297, 14)
Count  1 :  (886, 14)
Count  7 :  (541, 14)
Valid user =  5 , Extracted user =  2 accuracy =  0.7136772613533514
--------------------------------------------------------------------------------
True:  (6261, 14

True:  (716, 14)
Shape:  (2148, 14)
Count  8 :  (716, 14)
Count  1 :  (716, 14)
Count  5 :  (224, 14)
Count  4 :  (168, 14)
Count  6 :  (127, 14)
Count  2 :  (56, 14)
Count  7 :  (51, 14)
Count  3 :  (90, 14)
Valid user =  8 , Extracted user =  1 accuracy =  0.6666666666666666
--------------------------------------------------------------------------------
True:  (716, 14)
Shape:  (2148, 14)
Count  8 :  (716, 14)
Count  2 :  (716, 14)
Count  6 :  (139, 14)
Count  4 :  (141, 14)
Count  1 :  (84, 14)
Count  3 :  (88, 14)
Count  5 :  (209, 14)
Count  7 :  (55, 14)
Valid user =  8 , Extracted user =  2 accuracy =  0.6666666666666666
--------------------------------------------------------------------------------
True:  (716, 14)
Shape:  (2148, 14)
Count  8 :  (716, 14)
Count  3 :  (716, 14)
Count  5 :  (228, 14)
Count  4 :  (162, 14)
Count  2 :  (71, 14)
Count  1 :  (79, 14)
Count  6 :  (145, 14)
Count  7 :  (31, 14)
Valid user =  8 , Extracted user =  3 accuracy =  0.6666666666666666
----

Valid user =  3 , Extracted user =  2 accuracy =  0.7679324894514767
--------------------------------------------------------------------------------
True:  (158, 13)
Shape:  (474, 13)
Count  3 :  (158, 13)
Count  4 :  (158, 13)
Count  2 :  (20, 13)
Count  5 :  (66, 13)
Count  1 :  (32, 13)
Count  8 :  (15, 13)
Count  7 :  (10, 13)
Count  6 :  (15, 13)
Valid user =  3 , Extracted user =  4 accuracy =  0.7763713080168776
--------------------------------------------------------------------------------
True:  (158, 13)
Shape:  (474, 13)
Count  3 :  (158, 13)
Count  5 :  (158, 13)
Count  2 :  (21, 13)
Count  1 :  (34, 13)
Count  4 :  (56, 13)
Count  6 :  (21, 13)
Count  7 :  (12, 13)
Count  8 :  (14, 13)
Valid user =  3 , Extracted user =  5 accuracy =  0.8164556962025317
--------------------------------------------------------------------------------
True:  (158, 13)
Shape:  (474, 13)
Count  3 :  (158, 13)
Count  6 :  (158, 13)
Count  8 :  (9, 13)
Count  4 :  (54, 13)
Count  1 :  (28, 13)

True:  (102, 13)
Shape:  (306, 13)
Count  6 :  (102, 13)
Count  4 :  (102, 13)
Count  8 :  (11, 13)
Count  3 :  (25, 13)
Count  5 :  (29, 13)
Count  1 :  (16, 13)
Count  2 :  (11, 13)
Count  7 :  (10, 13)
Valid user =  6 , Extracted user =  4 accuracy =  0.7581699346405228
--------------------------------------------------------------------------------
True:  (102, 13)
Shape:  (306, 13)
Count  6 :  (102, 13)
Count  5 :  (102, 13)
Count  8 :  (7, 13)
Count  2 :  (7, 13)
Count  4 :  (52, 13)
Count  1 :  (15, 13)
Count  7 :  (7, 13)
Count  3 :  (14, 13)
Valid user =  6 , Extracted user =  5 accuracy =  0.761437908496732
--------------------------------------------------------------------------------
True:  (102, 13)
Shape:  (306, 13)
Count  6 :  (102, 13)
Count  7 :  (102, 13)
Count  4 :  (32, 13)
Count  5 :  (30, 13)
Count  1 :  (16, 13)
Count  8 :  (7, 13)
Count  3 :  (11, 13)
Count  2 :  (6, 13)
Valid user =  6 , Extracted user =  7 accuracy =  0.7189542483660131
----------------------